In [ ]:
# Standard library
from os.path import join

# Third-party
import corner
import astropy.units as u
import h5py
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

# Project
from thejoker import Paths
paths = Paths()
from thejoker.data import RVData
from thejoker.celestialmechanics import OrbitalParams
from thejoker.plot import plot_rv_curves, _truth_color

plt.style.use('../thejoker/thejoker.mplstyle')

from figurehelpers import samples_units, make_rv_curve_figure, apw_corner

In [ ]:
# experiment number
e_number = 2
e_name = 'undunc'

In [ ]:
data_filename = "../data/experiment{}.h5".format(e_number)

## read the data:

In [ ]:
# read the data
with h5py.File(data_filename, 'r') as f:
    data1 = RVData.from_hdf5(f['a'])
    data2 = RVData.from_hdf5(f['b'])
    truth_pars = OrbitalParams.from_hdf5(f['truth'])
    truth_vec = truth_pars.pack(plot_transform=True, units=samples_units)[0]

# read the samples from fixing the jitter
with h5py.File(join(paths.cache, 'experiment{}-fixed-jitter-a.h5'.format(e_number)), 'r') as g:
    pars1 = OrbitalParams.from_hdf5(g)
    samples1 = pars1.pack(plot_transform=True, units=samples_units)

# read the samples from sampling over the jitter
with h5py.File(join(paths.cache, 'experiment{}-fixed-jitter-b.h5'.format(e_number)), 'r') as g:
    pars2 = OrbitalParams.from_hdf5(g)
    samples2 = pars2.pack(plot_transform=True, units=samples_units)
    
# read the samples from sampling over the jitter
with h5py.File(join(paths.cache, 'experiment{}-sample-jitter-b.h5'.format(e_number)), 'r') as g:
    pars3 = OrbitalParams.from_hdf5(g)
    samples3 = pars3.pack(plot_transform=True, units=samples_units)

print("{} samples survived (a)".format(samples1.shape[0]))
print("{} samples survived (b)".format(samples2.shape[0]))
print("{} samples survived (c)".format(samples3.shape[0]))

In [ ]:
t_grid = np.linspace(data1.t.mjd.min(), data1.t.mjd.max(), 1024)
truth_rv = truth_pars.rv_orbit(0).generate_rv_curve(t_grid)

plt.plot(t_grid, truth_rv.to(samples_units['v0']).value, linestyle='--', 
         marker='', linewidth=1, alpha=0.9, color=_truth_color)
data1.plot(ax=plt.gca(), rv_unit=samples_units['v0'])

In [ ]:
fig = make_rv_curve_figure([data1, data2, data2], [pars1, pars2, pars3], 
                           truth_pars=truth_pars,
                           units=samples_units, rv_lim=(4, 12))
fig.axes[0].set_title("Experiment {}".format(e_number), fontsize=24)

fig.axes[0].text(55900, 10.8, "(a)", fontsize=18)
fig.axes[1].text(55900, 10.8, "(b)", fontsize=18)
fig.axes[2].text(55900, 10.8, "(c)", fontsize=18)

fig.tight_layout()
fig.savefig(join(paths.figures, '{}-rv-curves.pdf'.format(e_name)), dpi=128)

## Corner plots

In [ ]:
# Make a corner plot of all samples
_med_v0 = np.median(samples1[:,-1])
_mad_v0 = np.median(np.abs(samples1[:,-1] - _med_v0))
ranges = [(2.55, 6.45), (-0.025, 1.025), (0,360), (0,360), (2.5, 7.25),
          (0.5, 3.5), (_med_v0 - 5*_mad_v0, _med_v0 + 5*_mad_v0)]
labels = [r'$\ln \left(\frac{P}{\rm day}\right)$', '$e$', r'$\omega$ [deg]', r'$\phi_0$ [deg]',
          r'$\ln \left(\frac{s}{\rm m\,s^{-1}}\right)$', r'$K$ [km s$^{-1}$]', '$v_0$ [km s$^{-1}$]']

corner_style = dict(alpha=0.5, markersize=2., bins=16, color='#555555')

In [ ]:
# remove angles
a_idx = [2,3]

# used to remove jitter from samples
s_idx = 4

In [ ]:
_style = corner_style.copy()
_style['alpha'] = 0.1
_style['markersize'] = 1.

_idx = np.concatenate((a_idx, [s_idx]))
fig = apw_corner(np.delete(samples1, _idx, axis=1), 
                 range=np.delete(ranges, _idx, axis=0),
                 truths=np.delete(truth_vec, _idx), 
                 labels=np.delete(labels, _idx),
                 **_style)
fig.subplots_adjust(left=0.12, bottom=0.12, top=0.94)
fig.suptitle("Experiment {}(a): fixed jitter".format(e_number), fontsize=24)
fig.savefig(join(paths.figures, '{}-corner-a.pdf'.format(e_name)), dpi=128)

In [ ]:
_style = corner_style.copy()

_idx = np.concatenate((a_idx, [s_idx]))
fig = apw_corner(np.delete(samples2, _idx, axis=1), 
                 range=np.delete(ranges, _idx, axis=0),
                 truths=np.delete(truth_vec, _idx), 
                 labels=np.delete(labels, _idx),
                 **_style)
fig.subplots_adjust(left=0.12, bottom=0.12, top=0.94)
fig.suptitle("Experiment {}(b): fixed jitter, underestimated errors".format(e_number), fontsize=24)
fig.savefig(join(paths.figures, '{}-corner-b.pdf'.format(e_name)), dpi=128)

In [ ]:
_idx = a_idx
fig = apw_corner(np.delete(samples3, _idx, axis=1), 
                  range=np.delete(ranges, _idx, axis=0),
                  truths=np.delete(truth_vec, _idx), 
                  labels=np.delete(labels, _idx),
                  **corner_style)

fig.subplots_adjust(left=0.12, bottom=0.12, top=0.94)
fig.suptitle("Experiment {}(c): sample jitter, underestimated errors".format(e_number), fontsize=26)
fig.savefig(join(paths.figures, '{}-corner-c.pdf'.format(e_name)), dpi=128)